In [1]:
import cv2
import numpy as np

In [4]:
def show_images(lst):
    while True:
        for i in lst:
            cv2.imshow(i[0],i[1])
        ret = cv2.waitKey(0)
        if ret==27:
            break
    cv2.destroyAllWindows()

In [6]:
base_img = np.zeros((200,300),dtype=np.uint8)

In [7]:
base_img[50:150,50:150]=255

In [8]:
show_images([("baseimg",base_img)])

In [95]:
warp = np.float32([[0.5,0.5,0],[1,-0.202,0]])

In [96]:
res = cv2.warpAffine(base_img,warp,(300,200))

In [97]:
show_images([("res",res)])

In [98]:
IMG = res.copy()

## Detecting four corners of plane

In [65]:
projImg = cv2.imread("./image/img01.png")

In [74]:
show_images([("a",projImg)])

In [125]:
def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype = "float32")

    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    # return the ordered coordinates
    return rect

In [161]:
def get_corners_mask(mask):
    if len(mask.shape)>2:
        mask = cv2.cvtColor(mask,cv2.COLOR_BGR2GRAY)
    corners = cv2.goodFeaturesToTrack(mask,4,0.01,10)
    corners = np.int0(corners)
    corners = np.squeeze(corners)
    return order_points(corners)
    ## returns in order of (topl,bottoml,topr,bottomr)

In [146]:
def get_corners_img(img):
    ## in order of (topl,topr,bottomr,bottoml) in image coordinates
    return np.array([[0,0],[(img.shape[1]-1),0],
                        [(img.shape[1]-1),(img.shape[0]-1)],[0,(img.shape[0]-1)]],dtype=np.float32)

In [129]:
corners = get_corners_mask(IMG)

In [140]:
imgcorners = get_corners_img(projImg)

In [134]:
transform = cv2.getPerspectiveTransform(origcorners,corners)

## Testing

In [162]:
cap = cv2.VideoCapture("../projection/mask_skew_small.mp4")

In [163]:
imgcorners = get_corners_img(projImg)

In [164]:
while True:
    ret,mask = cap.read()
    if not ret:
        break
    corners = get_corners_mask(mask)
    transform = cv2.getPerspectiveTransform(imgcorners,corners)
    new_img = cv2.warpPerspective(projImg,transform,(mask.shape[1],mask.shape[0]))
    cv2.imshow("new_img",new_img)
    cv2.imshow("mask",mask)
    ret = cv2.waitKey(10)
    if(ret==27):
        break
cv2.destroyAllWindows()